In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date
import os
from datetime import datetime
import urllib2

In [60]:
df = pd.read_csv('C://Users//User//ppro8//3961//04102016.csv')
recent_price = df['price'][-100:].tolist()

In [65]:
def get_price(symbol):
    url = 'http://localhost:2628/GetLv1?symbol='+symbol
    response = urllib2.urlopen(url)
    data = response.read()
    price_point = data.find('LastPrice="')+len('LastPrice="')
    price_end = data.find('"',price_point)
    current_price = data[price_point:price_end]
    current_price = float(current_price)
    
    return current_price 

In [61]:
def buy_singal(path,symbol):
    df = pd.read_csv(path)
    min_price = 0
    df_last = df[-30:]
    min_price = df_last.price.min()
    
    current_price = get_price(symbol)
       
    if min_price == current_price:
        recent_price = df['price'][-30:].tolist()
        if recent_price.count(min_price) > 5: 
            return True, min_price

In [63]:
def buy(path,symbol):
    signal, buy_price = buy_singal(path,symbol)
    if signal == True:
            url = 'http://localhost:2628/ExecuteOrder?symbol='+symbol+'&limitprice='+buy_price+\
            '&ordername=JAPN Buy TSE Limit DAY&shares=100'
            response = urllib2.urlopen(url)
            
            return buy_price

In [66]:
def sell(symbol,price):
    url = 'http://localhost:2628/ExecuteOrder?symbol='+symbol+'&limitprice='+price+\
    '&ordername=JAPN Sell->Short TSE Limit DAY&shares=100'
    response = urllib2.urlopen(url)

In [67]:
def inventory(symbol):
    url = 'http://localhost:2628/GetOpenPositions?user=CHIFCHAN'
    response = urllib2.urlopen(url)
    data = response.read()
    if data.find(symbol) ==-1:
        return False
    else:
        return True

In [68]:
price_record = 0
def trade(path,symbol):
    inventory(symbol)
    if inventory == False:
        global price_record
        price_record = buy(path,symbol)
            
    if inventory == True:
        current_price = get_price(symbol)
        if price_record*0.995 >= current_price:
            sell(symbol,current_price)
        elif price_record*1.005 <= current_price:
            sell(symbol,current_price)